In [26]:
from damply import dirs
from collections import defaultdict
from rich import print
import pandas as pd
import tqdm
gdsqdir = dirs.RAWDATA / 'GDSC-Squared' / 'treatmentResponse'
files = list(gdsqdir.rglob('*.csv'))
print(files)



[
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/anchor/colon_anchor_combo.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/anchor/breast_anchor_combo.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/anchor/pancreas_anchor_combo.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/original_screen/Original_screen_All_tissues_raw_data.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/original_screen/Original_screen_All_tissues_day1_data.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/original_screen/Validation_screen_All_tissues_day1_data.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/original_screen/Validation_screen_All_tissues_raw_data.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/matrix/gdsc-008_matrix_results.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/matrix/gdsc-010_matrix_results.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/matrix/sandpiper-01_matrix_results.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/matrix/gdsc-007_matrix_results.csv'),
    PosixPath('/home/bioinf/bhklab/jermiah/projects/PharmacoSet-Pipelines/gdsc-treatmentresponse-snakemake/data/raw
data/GDSC-Squared/treatmentResponse/matrix/gdsc-009_matrix_results.csv')
]

In [27]:
df_dict = defaultdict(dict)
for f in tqdm.tqdm(files, desc="Processing files"):
    df_dict[str(f.parent.name)].update({
        f.name: pd.read_csv(f),
    })
    tqdm.tqdm.write(f"Loaded: {f.name}")

Processing files:   8%|▊         | 1/12 [00:00<00:02,  5.30it/s]

Loaded: colon_anchor_combo.csv


Processing files:  25%|██▌       | 3/12 [00:00<00:02,  4.25it/s]

Loaded: breast_anchor_combo.csv
Loaded: pancreas_anchor_combo.csv


Processing files:  33%|███▎      | 4/12 [00:13<00:42,  5.34s/it]

...

Loaded: Validation_screen_All_tissues_raw_data.csv
Loaded: gdsc-008_matrix_results.csv


Processing files:  92%|█████████▏| 11/12 [00:17<00:00,  1.37it/s]

Loaded: gdsc-010_matrix_results.csv
Loaded: sandpiper-01_matrix_results.csv
Loaded: gdsc-007_matrix_results.csv


Processing files: 100%|██████████| 12/12 [00:18<00:00,  1.50s/it]

Loaded: gdsc-009_matrix_results.csv


# Original Screen

In [ ]:
original_screen = df_dict['original_screen']
print({k: v.shape for k, v in original_screen.items()})

# there are two columns in two of the 4 files 

{
    'Original_screen_All_tissues_raw_data.csv': (8164070, 20),
    'Original_screen_All_tissues_day1_data.csv': (890880, 18),
    'Validation_screen_All_tissues_day1_data.csv': (402432, 18),
    'Validation_screen_All_tissues_raw_data.csv': (1106558, 20)
}

In [54]:
from dataclasses import dataclass, field
from pandas.api.types import infer_dtype
import numpy as np
@dataclass
class ColInfo:
    name: str
    type: str
    nunique: int
    first: str
    last: str

@dataclass
class DataFrameSummary:
    file: str
    shape: tuple
    # inferred type, nunique
    colinfo: list[ColInfo]

summaries = []
for fname, df in original_screen.items():
    summary = DataFrameSummary(
        file=fname,
        shape=df.shape,
        colinfo=[
            ColInfo(
                name=col,
                type=infer_dtype(df[col]),
                nunique=df[col].nunique(),
                first=df[col].iloc[0],
                last=df[col].iloc[-1],
            )
            for col in sorted(df.columns)
        ]
    )
    summaries.append(summary)
print(summaries)

[
    DataFrameSummary(
        file='Original_screen_All_tissues_raw_data.csv',
        shape=(8164070, 20),
        colinfo=[
            ColInfo(name='ASSAY', type='string', nunique=1, first='Glo', last='Glo'),
            ColInfo(name='BARCODE', type='integer', nunique=3106, first=np.int64(14482), last=np.int64(39392)),
            ColInfo(name='CELL_ID', type='integer', nunique=148, first=np.int64(6190), last=np.int64(7049)),
            ColInfo(name='CELL_LINE_NAME', type='string', nunique=125, first='HCC1806', last='PSN1'),
            ColInfo(name='CONC', type='floating', nunique=62, first=np.float64(nan), last=np.float64(nan)),
            ColInfo(name='COSMIC_ID', type='integer', nunique=125, first=np.int64(907047), last=np.int64(910546)),
            ColInfo(
                name='DATE_CREATED',
                type='string',
                nunique=109,
                first='2016-03-30T23:00:00Z',
                last='2018-05-21T23:00:00Z'
            ),
            ColInfo(name='DRUGSET_ID', type='integer', nunique=53, first=np.int64(264), last=np.int64(496)),
            ColInfo(name='DRUG_ID', type='floating', nunique=67, first=np.float64(nan), last=np.float64(nan)),
            ColInfo(name='DRUG_NAME', type='string', nunique=67, first=nan, last=nan),
            ColInfo(name='DURATION', type='integer', nunique=1, first=np.int64(4), last=np.int64(4)),
            ColInfo(name='INTENSITY', type='integer', nunique=5486, first=np.int64(0), last=np.int64(478)),
            ColInfo(name='MASTER_CELL_ID', type='integer', nunique=125, first=np.int64(1317), last=np.int64(560)),
            ColInfo(name='POSITION', type='integer', nunique=1536, first=np.int64(1), last=np.int64(1536)),
            ColInfo(name='RESEARCH_PROJECT', type='string', nunique=4, first='GDSC_Breast', last='GDSC_Panc'),
            ColInfo(name='SANGER_MODEL_ID', type='string', nunique=125, first='SIDM00875', last='SIDM00469'),
            ColInfo(
                name='SCAN_DATE',
                type='string',
                nunique=109,
                first='2016-04-03T23:00:00Z',
                last='2018-05-25T23:00:00Z'
            ),
            ColInfo(name='SCAN_ID', type='integer', nunique=3106, first=np.int64(14077), last=np.int64(38402)),
            ColInfo(name='SEEDING_DENSITY', type='integer', nunique=22, first=np.int64(325), last=np.int64(625)),
            ColInfo(name='TAG', type='string', nunique=165, first='UN-USED', last='UN-USED')
        ]
    ),
    DataFrameSummary(
        file='Original_screen_All_tissues_day1_data.csv',
        shape=(890880, 18),
        colinfo=[
            ColInfo(name='ASSAY', type='string', nunique=1, first='Glo', last='Glo'),
            ColInfo(name='BARCODE', type='integer', nunique=580, first=np.int64(14481), last=np.int64(39389)),
            ColInfo(name='CELL_ID', type='integer', nunique=148, first=np.int64(6190), last=np.int64(7049)),
            ColInfo(name='CELL_LINE_NAME', type='string', nunique=125, first='HCC1806', last='PSN1'),
            ColInfo(name='CONC', type='floating', nunique=0, first=np.float64(nan), last=np.float64(nan)),
            ColInfo(name='COSMIC_ID', type='integer', nunique=125, first=np.int64(907047), last=np.int64(910546)),
            ColInfo(
                name='DATE_CREATED',
                type='string',
                nunique=103,
                first='2016-03-30T23:00:00Z',
                last='2018-05-21T23:00:00Z'
            ),
            ColInfo(name='DRUGSET_ID', type='integer', nunique=1, first=np.int64(120), last=np.int64(120)),
            ColInfo(name='DRUG_ID', type='floating', nunique=0, first=np.float64(nan), last=np.float64(nan)),
            ColInfo(name='DURATION', type='integer', nunique=1, first=np.int64(4), last=np.int64(4)),
            ColInfo(name='INTENSITY', type='integer', nunique=3624, first=np.int64(0), last=np.int64(19780)),
            ColInfo(name='MASTER_CELL_ID', type='integer', nunique=125, first=np.int64(1317

In [53]:

common_cols = set.intersection(*[set(df.columns) for df in original_screen.values()])
print(f'There are {len(common_cols)} common columns: {common_cols}')
# see which files have uncommon columns
for fname, df in original_screen.items():
    uncommon_cols = set(df.columns) - common_cols
    if uncommon_cols:
        print(f'{fname} has uncommon columns: {uncommon_cols}')
    else:
        pass

There are 18 common columns: {'BARCODE', 'ASSAY', 'DURATION', 'SCAN_ID', 'POSITION', 'DRUG_ID', 'SCAN_DATE', 
'INTENSITY', 'TAG', 'MASTER_CELL_ID', 'CELL_LINE_NAME', 'DATE_CREATED', 'CELL_ID', 'CONC', 'RESEARCH_PROJECT', 
'SEEDING_DENSITY', 'DRUGSET_ID', 'COSMIC_ID'}

Original_screen_All_tissues_raw_data.csv has uncommon columns: {'DRUG_NAME', 'SANGER_MODEL_ID'}

Validation_screen_All_tissues_raw_data.csv has uncommon columns: {'DRUG_NAME', 'SANGER_MODEL_ID'}

In [58]:
df = original_screen['Original_screen_All_tissues_raw_data.csv']

# check if any entire column is empty
if any(df[col].isnull().all() for col in df.columns):
    print(f'There are empty columns in {fname}')



In [59]:
df.head()


,BARCODE,RESEARCH_PROJECT,SCAN_ID,DATE_CREATED,SCAN_DATE,CELL_ID,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,DRUGSET_ID,ASSAY,DURATION,POSITION,TAG,DRUG_ID,DRUG_NAME,CONC,INTENSITY
0,14482,GDSC_Breast,14077,2016-03-30T23:00:00Z,2016-04-03T23:00:00Z,6190,1317,907047,HCC1806,SIDM00875,325,264,Glo,4,1,UN-USED,NaN,NaN,NaN,0
1,14482,GDSC_Breast,14077,2016-03-30T23:00:00Z,2016-04-03T23:00:00Z,6190,1317,907047,HCC1806,SIDM00875,325,264,Glo,4,2,UN-USED,NaN,NaN,NaN,0
2,14482,GDSC_Breast,14077,2016-03-30T23:00:00Z,2016-04-03T23:00:00Z,6190,1317,907047,HCC1806,SIDM00875,325,264,Glo,4,3,UN-USED,NaN,NaN,NaN,764
3,14482,GDSC_Breast,14077,2016-03-30T23:00:00Z,2016-04-03T23:00:00Z,6190,1317,907047,HCC1806,SIDM00875,325,264,Glo,4,4,UN-USED,NaN,NaN,NaN,382
4,14482,GDSC_Breast,14077,2016-03-30T23:00:00Z,2016-04-03T23:00:00Z,6190,1317,907047,HCC1806,SIDM00875,325,264,Glo,4,5,UN-USED,NaN,NaN,NaN,287


# Anchor

In [61]:
anchor = df_dict['anchor']

anchorSummaries = []
for fname, df in anchor.items():
    summary = DataFrameSummary(
        file=fname,
        shape=df.shape,
        colinfo=[
            ColInfo(
                name=col,
                type=infer_dtype(df[col]),
                nunique=df[col].nunique(),
                first=df[col].iloc[0],
                last=df[col].iloc[-1],
            )
            for col in sorted(df.columns)
        ]
    )
    anchorSummaries.append(summary)
print(anchorSummaries)

[
    DataFrameSummary(
        file='colon_anchor_combo.csv',
        shape=(75400, 20),
        colinfo=[
            ColInfo(
                name=' Library Pathway',
                type='string',
                nunique=14,
                first='DNA replication',
                last='p53 pathway'
            ),
            ColInfo(
                name='Anchor Conc',
                type='floating',
                nunique=16,
                first=np.float64(0.625),
                last=np.float64(2.5)
            ),
            ColInfo(name='Anchor Name', type='string', nunique=26, first='Crizotinib', last='5-Fluorouracil'),
            ColInfo(name='Anchor Pathway', type='string', nunique=14, first='RTK signaling', last='Other'),
            ColInfo(
                name='Anchor Target',
                type='string',
                nunique=26,
                first='MET, ALK, ROS1',
                last='Antimetabolite (DNA & RNA)'
            ),
            ColInfo(
                name='Bliss Emax',
                type='floating',
                nunique=74681,
                first=np.float64(0.53474844),
                last=np.float64(0.8964314)
            ),
            ColInfo(
                name='Cancer Type',
                type='string',
                nunique=1,
                first='Colorectal Carcinoma',
                last='Colorectal Carcinoma'
            ),
            ColInfo(name='Cell Line name', type='string', nunique=45, first='RCM-1', last='CaR-1'),
            ColInfo(
                name='Combo Emax',
                type='floating',
                nunique=75272,
                first=np.float64(0.6542296),
                last=np.float64(0.9046309)
            ),
            ColInfo(
                name='Combo IC50',
                type='floating',
                nunique=75204,
                first=np.float64(10.796692),
                last=np.float64(15.115561)
            ),
            ColInfo(
                name='Delta Emax',
                type='floating',
                nunique=75360,
                first=np.float64(-0.1194811),
                last=np.float64(-0.00819951)
            ),
            ColInfo(
                name='Delta Xmid',
                type='floating',
                nunique=75359,
                first=np.float64(-1.3679672),
                last=np.float64(-0.2825557)
            ),
            ColInfo(
                name='Library Emax',
                type='floating',
                nunique=1508,
                first=np.float64(0.5394963),
                last=np.float64(0.9178527)
            ),
            ColInfo(
                name='Library IC50',
                type='floating',
                nunique=1508,
                first=np.float64(9.428724),
                last=np.float64(14.833005)
            ),
            ColInfo(name='Library Name', type='string', nunique=26, first='Camptothecin', last='Nutlin-3a (-)'),
            ColInfo(name='Maxc', type='floating', nunique=6, first=np.float64(0.1), last=np.float64(10.0)),
            ColInfo(name='SDIM', type='string', nunique=45, first='SIDM00359', last='SIDM00488'),
            ColInfo(name='Synergy?', type='integer', nunique=2, first=np.int64(0), last=np.int64(0)),
            ColInfo(name='Tissue', type='string', nunique=1, first='Large Intestine', last='Large Intestine'),
            ColInfo(name='library Target', type='string', nunique=26, first='TOP1', last='MDM2')
        ]
    ),
    DataFrameSummary(
        file='breast_anchor_combo.csv',
        shape=(163470, 20),
        colinfo=[
            ColInfo(
                name=' Library Pathway',
                type='string',
                nunique=20,
                first='DNA replication',
                last='Apoptosis regulation'
            ),
            ColInfo(
                name='Anchor Conc',
                type='floating',
                nunique=18,
                first=np.float64(0.25),
      

# Matrix

In [62]:
matrix = df_dict['matrix']
matrixSummaries = []
for fname, df in matrix.items():
    summary = DataFrameSummary(
        file=fname,
        shape=df.shape,
        colinfo=[
            ColInfo(
                name=col,
                type=infer_dtype(df[col]),
                nunique=df[col].nunique(),
                first=df[col].iloc[0],
                last=df[col].iloc[-1],
            )
            for col in sorted(df.columns)
        ]
    )
    matrixSummaries.append(summary)
print(matrixSummaries)

[
    DataFrameSummary(
        file='gdsc-008_matrix_results.csv',
        shape=(24941, 59),
        colinfo=[
            ColInfo(name='BARCODE', type='integer', nunique=1783, first=np.int64(42784), last=np.int64(52835)),
            ColInfo(
                name='Bliss_matrix',
                type='floating',
                nunique=24941,
                first=np.float64(-0.0030450878448877),
                last=np.float64(-0.0067606210045553)
            ),
            ColInfo(
                name='Bliss_matrix_SO',
                type='floating',
                nunique=24919,
                first=np.float64(0.0821686824583975),
                last=np.float64(0.0601569365611923)
            ),
            ColInfo(
                name='Bliss_synergistic_wells',
                type='integer',
                nunique=50,
                first=np.int64(17),
                last=np.int64(28)
            ),
            ColInfo(
                name='Bliss_window',
                type='floating',
                nunique=24941,
                first=np.float64(0.1737688303592666),
                last=np.float64(0.089034445552129)
            ),
            ColInfo(
                name='Bliss_window_SO',
                type='floating',
                nunique=24918,
                first=np.float64(0.1737688303592666),
                last=np.float64(0.1183978468595808)
            ),
            ColInfo(name='Bliss_window_SO_dose1', type='string', nunique=5, first='D5', last='D3'),
            ColInfo(name='Bliss_window_SO_dose2', type='string', nunique=5, first='D3', last='D3'),
            ColInfo(name='Bliss_window_dose1', type='string', nunique=5, first='D5', last='D1'),
            ColInfo(name='Bliss_window_dose2', type='string', nunique=5, first='D3', last='D1'),
            ColInfo(
                name='CANCER_TYPE',
                type='string',
                nunique=42,
                first='T-Lymphoblastic Leukemia',
                last='Breast Carcinoma'
            ),
            ColInfo(name='CELL_LINE_NAME', type='string', nunique=736, first='LOUCY', last='HCC1500'),
            ColInfo(
                name='COSMIC_ID',
                type='integer',
                nunique=736,
                first=np.int64(907789),
                last=np.int64(1303900)
            ),
            ColInfo(name='DRUGSET_ID', type='integer', nunique=4, first=np.int64(547), last=np.int64(570)),
            ColInfo(
                name='Delta_MaxE_lib1',
                type='floating',
                nunique=24936,
                first=np.float64(0.2909082650928463),
                last=np.float64(0.0657310626742194)
            ),
            ColInfo(
                name='Delta_MaxE_lib2',
                type='floating',
                nunique=24941,
                first=np.float64(0.1754521881822675),
                last=np.float64(0.6769254511872558)
            ),
            ColInfo(
                name='Delta_combo_MaxE_day1',
                type='floating',
                nunique=21937,
                first=np.float64(0.5012563354525342),
                last=np.float64(0.8577514858153016)
            ),
            ColInfo(
                name='HSA_matrix',
                type='floating',
                nunique=24941,
                first=np.float64(0.0418787838383904),
                last=np.float64(0.0223207264726719)
            ),
            ColInfo(
                name='HSA_matrix_SO',
                type='floating',
                nunique=24932,
                first=np.float64(0.0964217188604791),
                last=np.float64(0.0917667737670829)
            ),
            ColInfo(
                name='HSA_synergistic_wells',
                type='integer',
                nunique=50,
                first=np.int64(23),
                last=np.int64(30)
            ),
            ColInfo(
                name='HSA_window',
                type='floating',
                